## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer1

In [1]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
       
        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
         
        
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        

        
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        pass
        

- dev

In [2]:
# langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    # PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import DatetimeOutputParser
from langchain.output_parsers import OutputFixingParser

# base
import datetime as dt
import time

`create_history_question`

In [6]:
def create_history_question(topic):
    '''
    This method should output a historical question about the topic that has a date as the correct answer.
    For example:
    
        "On what date did World War 2 end?"
        
    '''

    # system
    system_template = ("You generate historical questions about {topic}, for which the answer is a date.\n"
                       "For example: 'On what date did World War 2 end?")
    system_prompt = SystemMessagePromptTemplate.from_template(system_template)

    # human
    human_template = "Generate a question"
    human_prompt = HumanMessagePromptTemplate.from_template(human_template)   

    # chat
    chat_prompt = ChatPromptTemplate([system_prompt, human_prompt])
    request = chat_prompt.format_prompt(topic=topic)

    # declare assistant
    chat = ChatOpenAI(model='gpt-4o-mini')

    # call assitant
    result = chat.invoke(request)

    # get question
    question = result.content

    return question

In [7]:
topic = "french revolution"

create_history_question(topic)

'On what date did the Storming of the Bastille occur, marking the beginning of the French Revolution?'

In [9]:
topic = "brazilian history"

create_history_question(topic)

'On what date did Brazil officially become a republic?'

In [11]:
topic = "aviation history"

create_history_question(topic) # fuck you

'On what date did the Wright brothers achieve the first powered flight?'

`get_AI_answer`

In [42]:
def get_AI_answer(question):
    '''
    This method should get the answer to the historical question from the method above.
    Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
    
    September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
    '''

    # output_parser
    date_parser = DatetimeOutputParser()

    # system
    system_template = "You correctly answer date related history questions"
    system_prompt = SystemMessagePromptTemplate.from_template(system_template)

    # human
    human_template = "{question}\n\n{format_instructions}"
    human_prompt = HumanMessagePromptTemplate.from_template(human_template)

    # chat
    chat_prompt = ChatPromptTemplate([system_prompt, human_prompt])
    request = chat_prompt.format_prompt(question=question,
                                        format_instructions=date_parser.get_format_instructions())
    
    # declare and call assistant
    chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    result = chat.invoke(request).content

    # parse result
    try:
        answer = date_parser.parse(result)
    except Exception as e:
        print("The answer was not correctly formatted:")
        print(e)
        raise

    return answer


In [43]:
topic = "brazilian history"

question = create_history_question(topic)
print(f"Q: {question}")

answer = get_AI_answer(question)
print(f"A: {answer}")

Q: On what date did Brazil gain independence from Portugal?
A: 1822-09-07 00:00:00


In [44]:
answer

datetime.datetime(1822, 9, 7, 0, 0)

`get_user_answer`

In [55]:
def get_user_answer(question):
    '''
    This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
    You can just use input() for this.
    '''

    print(f"Answer the question: {question}")
    
    try:
        year = int(input("Guess the year: "))
        month = int(input("Guess the month: "))
        day = int(input("Guess the day: "))

        user_datetime = dt.datetime(year, month, day)
    except:
        print("Incorrect input.")
        raise

    return user_datetime

In [59]:
# topic = "brazilian history"
# question = create_history_question(topic)

get_user_answer(question=question)

Answer the question: On what date did Brazil officially declare its independence from Portugal?


datetime.datetime(2020, 1, 1, 0, 0)

`check_user_answer`

In [66]:
def check_user_answer(user_answer, ai_answer):
    '''
    Should check the user answer against the AI answer and return the difference between them
    '''
    # print or return the difference between the answers here!
    
    days_off = (user_answer - ai_answer).days

    if days_off == 0:
        print("You nailed it! Congratulations!")
    else:
        print(f"Oops! You were wrong for {days_off} days =/")

In [68]:
user_answer = get_user_answer(question=question)
ai_answer = get_AI_answer(question)
(check_user_answer(user_answer=user_answer, ai_answer=ai_answer))

Answer the question: On what date did Brazil officially declare its independence from Portugal?
You nailed it! Congratulations!
None


`HistoryQuiz`

In [3]:
class HistoryQuiz():
    
    def create_history_question(self, topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''

        # system
        system_template = ("You generate a historical quiz question about {topic}, "
                        "for which the answer is a date. You only return the quiz.\n"
                        "For example: 'On what date did World War 2 end?")
        system_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # human
        human_template = "Generate a question where the correct answer is a date."
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)   

        # chat
        chat_prompt = ChatPromptTemplate([system_prompt, human_prompt])
        request = chat_prompt.format_prompt(topic=topic)

        # declare assistant
        chat = ChatOpenAI(model='gpt-4o-mini', temperature=1)

        # call assitant
        result = chat.invoke(request)

        # get question
        question = result.content

        return question
    
        
    def get_AI_answer(self, question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''

        # output_parser
        date_parser = DatetimeOutputParser()

        # system
        system_template = "You correctly answer date related history question. You answer just the date."
        system_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # human
        human_template = "{question}\n\n{format_instructions}"
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # chat
        chat_prompt = ChatPromptTemplate([system_prompt, human_prompt])
        request = chat_prompt.format_prompt(question=question,
                                            format_instructions=date_parser.get_format_instructions())
        
        # declare and call assistant
        chat = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        result = chat.invoke(request).content

        # parse result
        try:
            answer = date_parser.parse(result)
        except Exception as e:
            print("The answer was not correctly formatted:")
            print(e)
            raise

        return answer

    
    def get_user_answer(self, question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''

        print(f"Answer the question: {question}")
        
        try:
            year = int(input("Guess the year: "))
            month = int(input("Guess the month: "))
            day = int(input("Guess the day: "))

            user_datetime = dt.datetime(year, month, day)
        except:
            print("Incorrect input.")
            raise

        return user_datetime
        
        
    def check_user_answer(self, user_answer, ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        
        days_off = (user_answer - ai_answer).days

        print(f"Ohhhh you answered {user_answer:%Y-%m-%d}... is it correct?", end="\n\n", flush=True)
        for _ in range(3):
            time.sleep(.85)
            print(".", end="", flush=True)
        time.sleep(2)
        if days_off == 0:
            print("\n\nYou nailed it! Congratulations!")
        else:
            print(f"\n\nOops! You were wrong for {days_off} days... the correct answer was {ai_answer:%Y-%m-%d}")

    # ####PLAY
    def play(self, topic):

        question = self.create_history_question(topic)
        ai_answer = self.get_AI_answer(question)
        user_answer = self.get_user_answer(question)
        print()
        self.check_user_answer(user_answer=user_answer, ai_answer=ai_answer)

        

In [4]:
HistoryQuiz().play("charles darwin origin")

Answer the question: On what date did Charles Darwin publish "On the Origin of Species"?

Ohhhh you answered 1842-01-01... is it correct?

...

Oops! You were wrong for -6536 days... the correct answer was 1859-11-24
